In [1]:
from sqlalchemy import create_engine
import pandas as pd

In [2]:
# psycopg2 + sqlalchemy
engine = create_engine('postgresql+psycopg2://john@localhost/chinook')

## Track Table

In [3]:
media_type_raw = pd.read_sql_table('media_type', con=engine)
artist_raw = pd.read_sql_table('artist', con=engine)
track_raw = pd.read_sql_table('track', con=engine)
genre_raw = pd.read_sql_table('genre', con=engine)
album_raw = pd.read_sql_table('album', con=engine)

In [4]:
media_type_raw.rename(columns={'name': 'media_type'}, inplace=True)
artist_raw.rename(columns={'name': 'artist'}, inplace=True)
genre_raw.rename(columns={'name': 'genre'}, inplace=True)
album_raw.rename(columns={'title': 'album'}, inplace=True)

In [5]:
album = album_raw.merge(artist_raw, on='album_id')
track = track_raw.merge(media_type_raw, on='media_type_id')
track = track.merge(genre_raw, on='genre_id')
track_dim = track.merge(album, on='album_id')

In [6]:
track_dim.drop(['album_id', 'media_type_id', 'genre_id', 'artist_id'], axis=1, inplace=True)
track_dim.sort_values('track_id', inplace=True)

In [7]:
track_dim.head()

,track_id,name,composer,milliseconds,bytes,unit_price,media_type,genre,album,artist
0,1,For Those About To Rock (We Salute You),"Angus Young, Malcolm Young, Brian Johnson",343719,11170334,0.99,MPEG audio file,Rock,For Those About To Rock We Salute You,AC/DC
1246,2,Balls to the Wall,None,342562,5510424,0.99,Protected AAC audio file,Rock,Balls to the Wall,Accept
1247,3,Fast As a Shark,"F. Baltes, S. Kaufman, U. Dirkscneider & W. Ho...",230619,3990994,0.99,Protected AAC audio file,Rock,Restless and Wild,Accept
1248,4,Restless and Wild,"F. Baltes, R.A. Smith-Diesel, S. Kaufman, U. D...",252051,4331779,0.99,Protected AAC audio file,Rock,Restless and Wild,Accept
1249,5,Princess of the Dawn,Deaffy & R.A. Smith-Diesel,375418,6290521,0.99,Protected AAC audio file,Rock,Restless and Wild,Accept


In [9]:
track_dim.to_sql('track_dim', engine, index=False, method='multi')

## Customer table

In [10]:
customer_dim = pd.read_sql_table('customer', con=engine)

In [11]:
customer_dim.head()

,customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
0,1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
1,2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
2,3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3
3,4,Bjørn,Hansen,None,Ullevålsveien 14,Oslo,None,Norway,0171,+47 22 44 22 22,None,bjorn.hansen@yahoo.no,4
4,5,František,Wichterlová,JetBrains s.r.o.,Klanova 9/506,Prague,None,Czech Republic,14700,+420 2 4172 5555,+420 2 4172 5555,frantisekw@jetbrains.com,4


In [12]:
customer_dim.to_sql('customer_dim', engine, index=False, method='multi')

## Date Table

In [13]:
def create_date_table(start='2000-01-01', end='2020-12-31'):
    df = pd.DataFrame({'date': pd.date_range(start, end)})
    df['date_id'] = df.index + 1
    df['year'] = df.date.dt.year
    df['month'] = df.date.dt.month
    df['day'] = df.date.dt.day
    df['day_name'] = df.date.dt.weekday_name
    df['day_week'] = df.date.dt.dayofweek
    df['week'] = df.date.dt.weekofyear
    df['quarter'] = df.date.dt.quarter
    
    df = df[['date_id', 'date', 'year', 'month', 'day', 'day_name', 'day_week', 'week', 'quarter']] 
    
    return df

# for time_table

# def create_time_table(start='00:00', end='23:59', freq='1min'):
    
#     df = pd.DataFrame(pd.date_range(start, end, freq=freq ), columns=['datetime'])
#     df['time'] = df.datetime.dt.time
#     df['hour'] = df.datetime.dt.hour
#     df['minute'] = df.datetime.dt.minute

#     return df

In [14]:
date_dim = create_date_table()

In [15]:
date_dim.head()

,date_id,date,year,month,day,day_name,day_week,week,quarter
0,1,2000-01-01,2000,1,1,Saturday,5,52,1
1,2,2000-01-02,2000,1,2,Sunday,6,52,1
2,3,2000-01-03,2000,1,3,Monday,0,1,1
3,4,2000-01-04,2000,1,4,Tuesday,1,1,1
4,5,2000-01-05,2000,1,5,Wednesday,2,1,1


In [16]:
date_dim.to_sql('date_dim', engine, index=False, method='multi')

## InvoiceTable

In [17]:
invoice_raw = pd.read_sql_table('invoice', con=engine)

In [18]:
invoice_dim = invoice_raw.drop(['customer_id','invoice_date'], axis=1)

In [19]:
invoice_dim.head()

,invoice_id,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
0,1,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,1.98
1,2,Ullevålsveien 14,Oslo,None,Norway,0171,3.96
2,3,Grétrystraat 63,Brussels,None,Belgium,1000,5.94
3,4,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
4,5,69 Salem Street,Boston,MA,USA,2113,13.86


In [20]:
invoice_dim.to_sql('invoice_dim', engine, index=False, method='multi')

## Invoice Fact

In [21]:
invoice_line_raw = pd.read_sql_table('invoice_line', con=engine)

In [22]:
invoice = pd.merge(invoice_line_raw, invoice_raw[['invoice_id','invoice_date', 'customer_id']], on='invoice_id', how='left')
invoice = invoice.merge(date_dim[['date', 'date_id']], left_on='invoice_date', right_on='date')
invoice_fact = invoice[['invoice_line_id', 'invoice_id', 'track_id', 'date_id', 'customer_id', 'unit_price', 'quantity']]

In [23]:
invoice_fact.head()

,invoice_line_id,invoice_id,track_id,date_id,customer_id,unit_price,quantity
0,1,1,2,3289,2,0.99,1
1,2,1,4,3289,2,0.99,1
2,3,2,6,3290,4,0.99,1
3,4,2,8,3290,4,0.99,1
4,5,2,10,3290,4,0.99,1


In [24]:
invoice_fact.to_sql('invoice_fact', engine, index=False, method='multi')